In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

def view_database(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM gdp_by_country_data")
    rows = cursor.fetchall()

    print("Database contents:")
    for row in rows:
        print(row)

    conn.close()

def scrape_gdp_by_country():
    url = "https://www.worldometers.info/gdp/gdp-by-country/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr')
    gdp_by_country_data = []

    for row in rows:
        cells = row.find_all('td')
        if len(cells) >= 7:
            country = cells[1].text.strip()
            gdp = cells[2].text.strip().replace('$', '').replace(',', '')
            population = cells[5].text.strip().replace(',', '')
            gdp_per_capita = cells[6].text.strip().replace('$', '').replace(',', '')
            gdp_by_country_data.append((country, gdp, population, gdp_per_capita))
    
    return gdp_by_country_data



In [2]:
def create_database(gdp_by_country_data):
    conn = sqlite3.connect('gdp_by_country.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS gdp_by_country_data (
                        id INTEGER PRIMARY KEY, 
                        country TEXT UNIQUE, 
                        gdp REAL, 
                        population INTEGER, 
                        gdp_per_capita REAL
                    )''')
    
    for country, gdp, population, gdp_per_capita in gdp_by_country_data:
        try:
            cursor.execute("INSERT INTO gdp_by_country_data (country, gdp, population, gdp_per_capita) VALUES (?, ?, ?, ?)", (country, gdp, population, gdp_per_capita))
        except sqlite3.IntegrityError:
            # Handle duplicate data
            pass
    
    conn.commit()
    conn.close()

def main():
    gdp_by_country_data = scrape_gdp_by_country()
    if gdp_by_country_data:
        create_database(gdp_by_country_data)
    view_database("gdp_by_country.db")

if __name__ == "__main__":
    main()


Database contents:
(1, 'United States', 25462700000000.0, 338289857, 75269.0)
(2, 'China', 17963200000000.0, 1425887337, 12598.0)
(3, 'Japan', 4231140000000.0, 123951692, 34135.0)
(4, 'Germany', 4072190000000.0, 83369843, 48845.0)
(5, 'India', 3385090000000.0, 1417173173, 2389.0)
(6, 'United Kingdom', 3070670000000.0, 67508936, 45485.0)
(7, 'France', 2782910000000.0, 64626628, 43061.0)
(8, 'Russia', 2240420000000.0, 144713314, 15482.0)
(9, 'Canada', 2139840000000.0, 38454327, 55646.0)
(10, 'Italy', 2010430000000.0, 59037474, 34053.0)
(11, 'Brazil', 1920100000000.0, 215313498, 8918.0)
(12, 'Australia', 1675420000000.0, 26177413, 64003.0)
(13, 'South Korea', 1665250000000.0, 51815810, 32138.0)
(14, 'Mexico', 1414190000000.0, 127504125, 11091.0)
(15, 'Spain', 1397510000000.0, 47558630, 29385.0)
(16, 'Indonesia', 1319100000000.0, 275501339, 4788.0)
(17, 'Saudi Arabia', 1108150000000.0, 36408820, 30436.0)
(18, 'Netherlands', 991115000000.0, 17564014, 56429.0)
(19, 'Turkey', 905988000000.0, 

In [4]:
import pandas as pd

# Assuming "countries.csv" is stored in the "Data" folder relative to the current working directory
file_path = "Data/countries.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Rename the fourth column to "Country" and the fifth column to "Nationality"
df.rename(columns={df.columns[3]: "Country", df.columns[4]: "Nationality"}, inplace=True)

# Keep only the "Country" and "Nationality" columns
df = df[["Country", "Nationality"]]

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df


,Country,Nationality
0,Afghanistan,Afghan
1,Åland Islands,Åland Island
2,Albania,Albanian
3,Algeria,Algerian
4,American Samoa,American Samoan
5,Andorra,Andorran
6,Angola,Angolan
7,Anguilla,Anguillan
8,Antarctica,Antarctic
9,Antigua and Barbuda,Antiguan or Barbudan


In [18]:
matching_countries = df['Country'].isin(sql_df.values.flatten())

# Display the result
print("Matching countries between DataFrame and SQL database:")
print(matching_countries)

Matching countries between DataFrame and SQL database:
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37     True
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49     True
50     True
51     True
52     True
53     True
54     True
55     True
56     True
57     True
58     True
59     True
60     True
61     True
62     True
63     True
64     True
65     True
66     True
67     True
68     True
69     True
70     True
71     True
72     True
73     True
74     True
75     True
76     True
77     True
78     Tr

In [36]:
UPDATE gdp_by_country_data
SET United States = (
    SELECT Nationality
    FROM df
    WHERE df.United States = df.United States
);



SyntaxError: invalid syntax (3040490265.py, line 1)